In [1]:
import os
from pathlib import Path
import csv
import pandas as pd
import numpy as np
from sqlalchemy import Float, Text, Integer, Column, String
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, Session
from collections import OrderedDict
from math import sqrt
import logging, sys
logging.disable(sys.maxsize)

In [2]:
Base = declarative_base()

In [3]:
def get_file_names(folder):
    """
    """
    files_folders = {}
    for file in folder.iterdir():
        
        # Stripping out the file names from the rest of the folder string
        file_name = str(file).lower()
        base_name = os.path.basename(file_name)
        
        if base_name.endswith('.csv'):
            file_name = base_name.split('.')[0] # Do not take the file .csv extension     
        if file_name in ['train','test','ideal']:
            _ = {file_name : str(file)}
            files_folders[file_name] = _
        
    check_file_count = len(files_folders)
    
    try:
        assert check_file_count == 3
    except:
        print('There are missing files - please load all 3 files')
    
    return files_folders

In [4]:
# add to  name == main
folder = Path('./Documents/Datasets_written_assignment')
files_folder = get_file_names(folder)

In [5]:
def df_create(table_name):
    """
    """
    conn = engine.connect()
    stmt = "SELECT * FROM " + table_name
    e_stmt = conn.execute(stmt)
    conn.close()
    df = pd.DataFrame(e_stmt.fetchall())
    df.columns = e_stmt.keys()
    df.set_index('x', inplace=True)
    df.sort_index(inplace=True)
    df = df.drop(['index'], axis=1)
    return df

In [6]:
engine = create_engine("sqlite://", echo=True)

In [7]:
def table_conversion(files_folder):
    """
    """
    for file_name, file_path in files_folder.items():        
        # Reading in the file
        __data = []
        with open(file_path) as csv_file:
            csv_reader = csv.DictReader(csv_file)
            for rows in map(dict, csv_reader):
                rows.update({'name' : file_name})
                __data.append(rows)

            _data = []
            for line in __data:
                _ = {}
                for key, val in line.items():
                    if key == 'name':
                        _[key] = val
                    else:
                        _[key] = float(val)
                _data.append(_)

            # Renaming y columns to be aligned with file name
            _key_lookups = {}
            for key in _data[0].keys():
                if 'y' in key:
                    _key_lookups[key] = '_'.join((key, _data[0]['name'], 'func'))
                else:
                    _key_lookups[key] = key

            converted_data = []
            for line in _data:
                _ = {}
                for key, value in _key_lookups.items():
                    _.update({value : line[key]})
                converted_data.append(_)  
            return converted_data

In [8]:
test = files_folder['test']
train = files_folder['train']
ideal = files_folder['ideal']

In [9]:
test_d, train_d, ideal_d = table_conversion(test), table_conversion(train), table_conversion(ideal)

In [10]:
class SQLTableBuilder:
    def __init__(self, dict_file):
        self.dict_file = dict_file
    
    def index_create(self):
        """
        Adds additional index column to data dictionary to satisfy unique column requirement.
        Input:
        Output:
            Altered dictionary
        """
        _, _converted_data = [], []

        for i in range(len(self.dict_file)):
            _.append({'index' : i})

        for dict_line, list_line in zip(self.dict_file, _):
            dict_line.update(list_line)
            _converted_data.append(dict_line)

        converted_data = []
        for dict_item in _converted_data:
            _dict = {}
            for key, value in dict_item.items():
                if 'y' in key and 'delta' not in key:
                    num = key[1:].split('_')[0]
                    re1, re2 = key[1:].split('_')[1], key[1:].split('_')[2]
                    num = str(num)
                    if len(num) == 1:
                        num = '0'+ num
                    y_num = 'y' + num
                    k = '_'.join((y_num, re1, re2))
                else:
                    k = key
                _dict[k] = value
                _ = OrderedDict(sorted(_dict.items()))
            converted_data.append(_)

        return converted_data
    
    def schema_create(self):
        """
        """
        _dict = dict(self.index_create()[0])
        key = _dict['name']

        file_name = key.title()

        clsdict = {"clsname" : ''.join(('My', file_name, 'Table'))}
        clsdict['__tablename__'] = file_name 
        clsdict['__table_args__'] = {'extend_existing': True}

        data_to_load = []
        for line in self.index_create():
            data_d = {}
            for key, value in line.items():       
                if key == 'index':
                    _ = {key : Column(Integer, primary_key=True, unique=True)}
                elif key == 'num_of_ideal_func':
                    _ = {key : Column(String)}
                elif key == 'name':
                    continue
                else:
                    _ = {key : Column(Float)}
                clsdict.update(_)    
                data_d.update({key: value})
            data_to_load.append(data_d)

        return clsdict, data_to_load
    
    def class_constructor(self):
        MyClass = type(self.schema_create()[0]['clsname'], (Base,), self.schema_create()[0])
        Base.metadata.create_all(engine)
        return MyClass

In [11]:
# del
train_ins = SQLTableBuilder(train_d)

In [12]:
# del
train_ins.schema_create()[0]['clsname']

'MyTrainTable'

In [13]:
# del
train_ins.schema_create()[0]

{'clsname': 'MyTrainTable',
 '__tablename__': 'Train',
 '__table_args__': {'extend_existing': True},
 'index': Column(None, Integer(), table=None, primary_key=True, nullable=False),
 'x': Column(None, Float(), table=None),
 'y01_train_func': Column(None, Float(), table=None),
 'y02_train_func': Column(None, Float(), table=None),
 'y03_train_func': Column(None, Float(), table=None),
 'y04_train_func': Column(None, Float(), table=None)}

In [14]:
train_ins = SQLTableBuilder(train_d)
MyTrainClass = train_ins.class_constructor()
_, data_train = train_ins.schema_create()

In [15]:
test_ins = SQLTableBuilder(test_d)
MyTestClass = test_ins.class_constructor()
_, data_test = test_ins.schema_create()

In [16]:
ideal_ins = SQLTableBuilder(ideal_d)
MyIdealClass = ideal_ins.class_constructor()
_, data_ideal = ideal_ins.schema_create()

In [17]:
with Session(engine) as sess:
    sess.add_all(MyIdealClass(**rec) for rec in data_ideal)
    sess.add_all(MyTestClass(**rec) for rec in data_test)
    sess.add_all(MyTrainClass(**rec) for rec in data_train)
    sess.commit() 

In [18]:
train = df_create('Train')

In [19]:
test = df_create('Test')

In [20]:
ideal = df_create('Ideal')

In [21]:
class TrainFunctionReturner:
    def __init__(self, train_df):
        self.train_df = train_df
        #self.sq_root_number = sq_root_number
    
    def ideal_function(self):
        """
        """
        # Calculates the sum of squares per column of the ideal functions
        sum_squares = []    
        for column in self.train_df.columns:
            column_name, _sum_sq_diff = [], []
            for col in ideal.columns:
                diff = self.train_df[column] - ideal[col]
                sq_diff = diff ** 2
                sq_diff = sq_diff.fillna(0)
                sum_sq_diff = sum(sq_diff)
                column_name.append(col)
                _sum_sq_diff.append(sum_sq_diff)
            sum_squares.append([column_name, _sum_sq_diff])
        
        # Retrieves the ideal functions per column of self.train_df
        top_ = []
        for ls_data in sum_squares:
            zipped = zip(ls_data[0], ls_data[1])
            zipped = list(zipped)
            sorted_zip = sorted(zipped, key = lambda x: x[1])
            top_fns = sorted_zip[0][0]
            top_.append(top_fns)
        
        # Outputs the results as a dictionary
        output_dict = {column : list_entry for (column, list_entry) in zip(self.train_df.columns, top_)}    
        return output_dict
            
    def mapped_fns(self):
        """
        """
        
        df_list = []
        for key, value in self.ideal_function().items():
            
            # Finds the largest deviation between the train x,y and ideal x,y co-ords.
            # Then multiplies the largest deviation by the sqrt of 2 (or inputted number)
            larg_dev = ((self.train_df[key] - ideal[value]).max()) * sqrt(2)
            
            # Adds and subtracts max_diff from the ideal function to give 
            # the upper & lower bounds per x, y co-ordinate within the ideal fn.
            upper = pd.Series(ideal[value] + larg_dev, name = 'upper_bound')
            lower = pd.Series(ideal[value] - larg_dev, name = 'lower_bound')
            ideal_d = pd.Series(ideal[value], name = 'y_ideal')
            
            # Returning a Pandas dataframe with results
            _df = pd.concat([ideal_d, upper, lower], axis=1)
            _df['max_dev'] = larg_dev
            _df['name'] = ideal[value].name
            df_list.append(_df)
        return df_list 

In [22]:
a = TrainFunctionReturner(train)

In [23]:
a.mapped_fns()[0].head()

,y_ideal,upper_bound,lower_bound,max_dev,name
x,,,,,
-20.0,-8020.000,-8019.303358,-8020.696642,0.696642,y28_ideal_func
-19.9,-7900.499,-7899.802358,-7901.195642,0.696642,y28_ideal_func
-19.8,-7782.192,-7781.495358,-7782.888642,0.696642,y28_ideal_func
-19.7,-7665.073,-7664.376358,-7665.769642,0.696642,y28_ideal_func
-19.6,-7549.136,-7548.439358,-7549.832642,0.696642,y28_ideal_func


In [24]:
class TestFunctionReturner(TrainFunctionReturner):
    
    # Initiating new constructor
    def __init__(self, train_df, test_df):
        
        # Calling parent's (TrainFunctionReturner) constructor
        super().__init__(train_df)
        
        # Assigning the test_df attribute
        self.test_df = test_df
        
    def mapped_fns_df(self):
        """
        """
        mapped_fn_df = super().mapped_fns()
        _output_list = []
        
        # Checking the deviation of the test_df to see if it lies between the lower and 
        # upper bounds of the ideal function
        for line in self.test_df.itertuples():
            for ideal_fn in mapped_fn_df:
                i = []
                for row in ideal_fn.itertuples():
                    if line.Index == row.Index:
                        diff = line.y_test_func - row.y_ideal
                        abs_diff = abs(diff)
                        #sq_diff = diff ** 2
                        if abs_diff <= row.max_dev:
                            i.append([line.Index, line.y_test_func, abs_diff, row.name])
                _output_list.append(i)       
        output_list = [tuple(values) for line in _output_list for values in line]
        
        # Returning a Pandas dataframe with results
        mapped_output_df = pd.DataFrame(output_list, columns=['x', 'y_test_func', 'delta_y_test_func', 'num_of_ideal_func'])
        mapped_output_df.set_index('x', inplace=True)
        mapped_output_df.sort_index(inplace=True)
        return mapped_output_df
    
    def mapped_fns_dict(self):
        """
        """
        _mapped_fns_dict = self.mapped_fns_df().to_dict('records')
        
        _list = []
        for x_val in self.mapped_fns_df().index:
            _dict = {}
            _dict['x'] = x_val
            _dict['name'] = 'mapped'
            _list.append(_dict)
        
        for dict_entries, x_val in zip(_mapped_fns_dict, _list):
            dict_entries.update(x_val)
        return _mapped_fns_dict
        
    def unmapped_fns(self): 
        """
        """
        # Adding remaining x and y coordinates which do not have a corresponding ideal function
        out = set(self.mapped_fns_df().index.values)
        idx = set(self.test_df.index.values)
        dif = idx.difference(out)
        
        output_list = []
        for item in dif:
            y_ = self.test_df.loc[self.test_df.index == item, 'y_test_func'].values.tolist()
            for data in y_:
                _output = (item, data)
                output_list.append(_output)
                
        # Returning a Pandas dataframe with results        
        unmapped_output_df = pd.DataFrame(output_list, columns=['x', 'y_test_func'])
        unmapped_output_df.set_index('x', inplace=True)
        unmapped_output_df.sort_index(inplace=True)
        return unmapped_output_df
    
    def all_fns(self):
        """
        """
        return self.mapped_fns_df().append(self.unmapped_fns())

In [25]:
b = TestFunctionReturner(test_df=test, train_df=train)

In [26]:
all_fns = b.all_fns()

In [27]:
all_fns.head()

,y_test_func,delta_y_test_func,num_of_ideal_func
x,,,
-19.5,-1.444449,0.496869,y07_ideal_func
-17.9,-0.769047,0.120348,y07_ideal_func
-17.2,-5105.398000,0.250000,y28_ideal_func
-15.9,-0.340016,0.244145,y07_ideal_func
-15.9,-0.340016,0.246340,y48_ideal_func


In [28]:
mapped_d = b.mapped_fns_dict()

In [29]:
mapped_d[0]

{'y_test_func': -1.4444486,
 'delta_y_test_func': 0.4968688000000001,
 'num_of_ideal_func': 'y07_ideal_func',
 'x': -19.5,
 'name': 'mapped'}

In [30]:
mapped_ins = SQLTableBuilder(mapped_d)
MyMappedClass = mapped_ins.class_constructor()
_, data_mapped = mapped_ins.schema_create()

In [31]:
with Session(engine) as sess:
    sess.add_all(MyMappedClass(**rec) for rec in data_mapped)
    sess.commit() 

In [32]:
mapped_fn = df_create('Mapped')

In [33]:
mapped_fn.head()

,delta_y_test_func,num_of_ideal_func,y_test_func
x,,,
-19.5,0.496869,y07_ideal_func,-1.444449
-17.9,0.120348,y07_ideal_func,-0.769047
-17.2,0.250000,y28_ideal_func,-5105.398000
-15.9,0.244145,y07_ideal_func,-0.340016
-15.9,0.246340,y48_ideal_func,-0.340016


In [34]:
b.mapped_fns_df().shape

(60, 3)

In [35]:
b.all_fns()['num_of_ideal_func'].unique()

array(['y07_ideal_func', 'y28_ideal_func', 'y48_ideal_func',
       'y05_ideal_func', nan], dtype=object)

In [36]:
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import row
from bokeh.palettes import Spectral11

In [37]:
class IdealPlotter(TestFunctionReturner):
    def __init__(self, train_df, test_df):
        
        # Calling parent's (TrainFunctionReturner) constructor
        super().__init__(train_df, test_df)
    
    def separate_plotted_fns(self):
        """
        """
        all_fns = super().all_fns()
        mf = super().mapped_fns()
        
        plot_df = all_fns[['num_of_ideal_func', 'y_test_func']]
        plot_df['num_of_ideal_func'].fillna('None', inplace=True)  
        fn_list = list(plot_df['num_of_ideal_func'].unique())             
        num_lines = len(fn_list)
        
        palette = Spectral11[0:5]
        
        for fn_num in range(num_lines - 1):
            borders_to_plot = mf[fn_num][['upper_bound', 'lower_bound']]
            fn_to_plot = mf[fn_num]['y_ideal']
            name = mf[fn_num]['name'].unique()[0]
            title_label = name.replace('_',' ') + ' plotted with upper and lower bounds and test points'
            p = figure(width=750, 
                       height=750, 
                       x_axis_label='x', 
                       y_axis_label='y', 
                       title=title_label,
                      title_location='above')
            p.title.text_font_size = "18px"
            p.multi_line(xs=[borders_to_plot.index.values] * 2,
                        ys=[borders_to_plot[col_name] for col_name in borders_to_plot.columns],
                        line_color=palette[fn_num],
                        line_width=4)
            p.line(x=fn_to_plot.index,
                  y=fn_to_plot.values,
                  line_color=palette[fn_num + 1],
                  line_width=2,
                  legend_label=name.replace('_',' '))
            
            matched_fn_df = all_fns[all_fns['num_of_ideal_func'] == name]
            p.circle(x=matched_fn_df.index,
                     y=matched_fn_df['y_test_func'].values,
                     size=3,
                     color="#000000", 
                     alpha=0.8)
            show(p)

In [38]:
x = IdealPlotter(train, test)

In [39]:
x.separate_plotted_fns()

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:6383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
